# Data Modeling with Apache Cassandra

The following notebook is an adaptation of the initial template that was supplied.  Deviations from the initial notebook will be noted below.

In [1]:
import main
import cql
import config
import os
import glob
import pandas as pd
import re
import matplotlib.pyplot as plt

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
%matplotlib inline

## Part 1 - Pre-processing

In the initial notebook, pre-processing was done over several steps:

1. Creating a list of files from a filepath
2. Iterating over each file and appending each row to a list
3. Create larger csv file from created list

The new adaptation does the following:

1. Construct a file directory where files reside
2. Create list of files from the file directory
3. Construct a pandas dataframe from the list of files

The code below (without docstrings) is taken out of `main.py` and shows the steps above.

In [2]:
def get_file_directory(filepath):
    cwd = os.getcwd()
    cwd += filepath
    return cwd

def get_files(file_directory, format="csv"):
    return glob.glob(file_directory + f"/*.{format}")


def construct_dataframe_from_files(files):
    df = pd.concat((pd.read_csv(f) for f in files))
    df.dropna(subset=['artist'], inplace=True)
    df[['itemInSession', 'sessionId', 'userId']] = \
        df[['itemInSession', 'sessionId', 'userId']].apply(
            pd.to_numeric, downcast='integer')
    return df

The code above is utilized inside a `run` function.  The `run` function is passed a filepath from an argument from the command line or, if nothing is provided, it will default to `/event_data`.  The first part of the function is as follows:

In [3]:
file_directory = get_file_directory('/event_data')
files = get_files(file_directory)
dataframe = construct_dataframe_from_files(files)

In [4]:
print("There are {} rows in the dataframe".format(len(dataframe)))
dataframe.head(5)

There are 6820 rows in the dataframe


,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userId
0,A Fine Frenzy,Logged In,Anabelle,F,0,Simpson,267.91138,free,"Philadelphia-Camden-Wilmington, PA-NJ-DE-MD",PUT,NextSong,1.541040e+12,256,Almost Lover (Album Version),200,1.541380e+12,69
1,Nirvana,Logged In,Aleena,F,0,Kirby,214.77832,paid,"Waterloo-Cedar Falls, IA",PUT,NextSong,1.541020e+12,237,Serve The Servants,200,1.541380e+12,44
2,Television,Logged In,Aleena,F,1,Kirby,238.49751,paid,"Waterloo-Cedar Falls, IA",PUT,NextSong,1.541020e+12,237,See No Evil (Remastered LP Version),200,1.541380e+12,44
3,JOHN COLTRANE,Logged In,Aleena,F,2,Kirby,346.43546,paid,"Waterloo-Cedar Falls, IA",PUT,NextSong,1.541020e+12,237,Blues To Bechet (LP Version),200,1.541380e+12,44
4,NOFX,Logged In,Aleena,F,3,Kirby,80.79628,paid,"Waterloo-Cedar Falls, IA",PUT,NextSong,1.541020e+12,237,It's My Job To Keep Punk Rock Elite,200,1.541380e+12,44


Part of the project's requirements is to create an event_data_new.csv file.  I won't use this for my ETL process, but the code below will create the file

In [5]:
dataframe.to_csv('event_datafile_new.csv', index=False)

## Part 2 - Apache Cassandra

The second part of the project is to create tables and insert data into an Apache Cassandra Cluster.  In an effort to reduce code duplication, I created two classes:  `Cassandra` and `ETL`.  Cassandra functions as my connection to the Cluster and is the base object in the ETL class.  ETL functions as my means to execute queries (CREATE and INSERT).  ETL relies on a dictionary, which can be found in the `config.py` file, as well as a dataframe, which serves as the data to be inserted into the database.

In [6]:
# Initialize ETL class
etl = cql.ETL(config.sparkify_dictionary, dataframe)

# Inspect Class
etl.cluster
etl.session
etl.dictionary
etl.dataframe.head(5)

{'item': {'table': 'item_detail',
  'columns': ['sessionId', 'itemInSession', 'artist', 'song', 'length'],
  'column_types': ['int', 'int', 'text', 'text', 'float'],
  'primary_key': ['sessionId', 'itemInSession'],
  'primary_key_description': '\n            The item_detail table uses a compound primary key with two components:\n            sessionId and itemInSession.  The sessionId will be how the data\n            is partitioned in the cluster and itemInSession will (1)\n            enforce uniqueness on the row and (2) order the data in ascending\n            order.  The query used for this table is concerned with a specific\n            item in a session so it made sense to me to partition the data by\n            session.\n        '},
 'session': {'table': 'session_detail',
  'columns': ['sessionId',
   'itemInSession',
   'artist',
   'song',
   'userId',
   'firstName',
   'lastName'],
  'column_types': ['int', 'int', 'text', 'text', 'int', 'text', 'text'],
  'primary_key': ['u

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userId
0,A Fine Frenzy,Logged In,Anabelle,F,0,Simpson,267.91138,free,"Philadelphia-Camden-Wilmington, PA-NJ-DE-MD",PUT,NextSong,1.541040e+12,256,Almost Lover (Album Version),200,1.541380e+12,69
1,Nirvana,Logged In,Aleena,F,0,Kirby,214.77832,paid,"Waterloo-Cedar Falls, IA",PUT,NextSong,1.541020e+12,237,Serve The Servants,200,1.541380e+12,44
2,Television,Logged In,Aleena,F,1,Kirby,238.49751,paid,"Waterloo-Cedar Falls, IA",PUT,NextSong,1.541020e+12,237,See No Evil (Remastered LP Version),200,1.541380e+12,44
3,JOHN COLTRANE,Logged In,Aleena,F,2,Kirby,346.43546,paid,"Waterloo-Cedar Falls, IA",PUT,NextSong,1.541020e+12,237,Blues To Bechet (LP Version),200,1.541380e+12,44
4,NOFX,Logged In,Aleena,F,3,Kirby,80.79628,paid,"Waterloo-Cedar Falls, IA",PUT,NextSong,1.541020e+12,237,It's My Job To Keep Punk Rock Elite,200,1.541380e+12,44


The ETL class also has attributes that are accessed when given a key of the dictionary.

In [7]:
key = list(etl.dictionary.keys())[0]
f"Columns for {key}"
etl._columns(key)
etl._column_types(key)
etl._column_placeholders(key)
etl._primary_key(key)
etl._create_statement(key)
etl._insert_statement(key)

'Columns for item'

['sessionId', 'itemInSession', 'artist', 'song', 'length']

['int', 'int', 'text', 'text', 'float']

'%s, %s, %s, %s, %s'

['sessionId', 'itemInSession']

'\n            CREATE TABLE IF NOT EXISTS item_detail\n            (sessionId int, itemInSession int, artist text, song text, length float,\n            PRIMARY KEY (\n                sessionId, itemInSession))\n        '

'\n            INSERT INTO item_detail\n            (sessionId, itemInSession, artist, song, length)\n            VALUES (%s, %s, %s, %s, %s)\n        '

The main access point into the `ETL` class is through the `run` method.  This method will take care of creating the table (if it doesn't exist) and then inserting the appropriate data.

### Important Caveat
The columns specified in the dictionary need to be the **EXACT SAME** as what's in the supplied dataframe

In [8]:
for key in etl.dictionary.keys():
    etl.run(key)
    print(f"Data for {etl._table(key)} inserted")
    print(f"Primary Key rationale for {etl._table(key)}:")
    print(etl.dictionary[key]['primary_key_description'])

Data for item_detail inserted
Primary Key rationale for item_detail:

            The item_detail table uses a compound primary key with two components:
            sessionId and itemInSession.  The sessionId will be how the data
            is partitioned in the cluster and itemInSession will (1)
            enforce uniqueness on the row and (2) order the data in ascending
            order.  The query used for this table is concerned with a specific
            item in a session so it made sense to me to partition the data by
            session.
        
Data for session_detail inserted
Primary Key rationale for session_detail:

            The session_detail table uses a compound primary key with three
            components:  userId, sessionId, and itemInSession.  The userId will
            be how the data is partitioned in the cluster and sessionId and
            itemInSession will (1) enforce uniqueness on the row and (2) order
            the data in ascending order.  The que

In [9]:
# Verify data has been inserted into the database
item_detail_query = "SELECT artist, song, length FROM item_detail WHERE sessionId = 338 AND itemInSession = 4"
session_detail_query = "SELECT artist, song, firstName, lastName FROM session_detail WHERE userId = 10 AND sessionId = 182"
song_detail_query = "SELECT firstName, lastName FROM song_detail WHERE song = 'All Hands Against His Own'"
queries = [item_detail_query, session_detail_query, song_detail_query]
for query in queries:
    rows = etl.session.execute(query)
    table = re.search('FROM (.+?) WHERE', query)
    print("Data from {}".format(table.group(1)))
    for row in rows:
        print(row)

Data from item_detail
Row(artist='Faithless', song='Music Matters (Mark Knight Dub)', length=495.30731201171875)
Data from session_detail
Row(artist='Down To The Bone', song="Keep On Keepin' On", firstname='Sylvie', lastname='Cruz')
Row(artist='Three Drives', song='Greece 2000', firstname='Sylvie', lastname='Cruz')
Row(artist='Sebastien Tellier', song='Kilometer', firstname='Sylvie', lastname='Cruz')
Row(artist='Lonnie Gordon', song='Catch You Baby (Steve Pitron & Max Sanna Radio Edit)', firstname='Sylvie', lastname='Cruz')
Data from song_detail
Row(firstname='Sara', lastname='Johnson')
Row(firstname='Jacqueline', lastname='Lynch')
Row(firstname='Tegan', lastname='Levine')


In [10]:
# Drop tables before closing out session
tables = [etl.dictionary[key]['table'] for key in etl.dictionary.keys()]

for table in tables:
    try:
        query = f'drop table if exists {table}'
        etl.session.execute(query)
    except Exception as e:
        print(e)

In [11]:
# Shutdown session and cluster
etl.session.shutdown()
etl.cluster.shutdown()